In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf

hello world


#### 卷积层

In [3]:
#1.滤波器尺寸（5*5） 2.当前层的深度（3） 3.滤波器个数（16）。
filter_weight = tf.get_variable('weight', [5, 5, 3, 16],
                                initializer=tf.truncated_normal_initializer(stddev=0.1))
biases=tf.get_variable('biases',[16],initializer=tf.constant_initializer(0.1))

conv=tf.nn.conv2d(input,filter_weight,strides=[1,1,1,1],padding='SAME')
bias=tf.nn.bias_add(conv,biases)

#激活
actived_conv=tf.nn.relu(bias)


Instructions for updating:
Colocations handled automatically by placer.


TypeError: Failed to convert object of type <class 'method'> to Tensor. Contents: <bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x7f86d8a476d0>>. Consider casting elements to a supported type.

#### 池化层

In [ ]:
pool=tf.nn.max_pool(actived_conv,ksize=[1,3,3,1],strides=[1,2,2,1],padding='SAME')



#### Inception-v3模块

In [4]:
slim=tf.contrib.slim

#设置默认的参数值
with slim.arg_scope([slim.conv2d,slim.max_pool2d,slim.avg_pool2d],stride=1,padding='VALID'):
    net='上一层的输出矩阵'
    with tf.variable_scope('Mixed_7c'):
        #第一条路径
        with tf.variable_scope("Branch_0"):
            branch_0=slim.conv2d(net,320,[1,1],scope='Conv2d_0a_1x1')

        #第二条路径
        with tf.variable_scope("Branch_1"):
            branch_1=slim.conv2d(net,384,[1,1],scope='Conv2d_0a_1x1')
            branch_1=tf.concat(3,[slim.conv2d(branch_1,384,[1,3],scope='Conv2d_0b_1x3'),
                                  slim.conv2d(branch_1,384,[3,1],scope='Conv2d_0c_3x1')])

        #第三条路径
        with tf.variable_scope("Branch_2"):
            branch_2=slim.conv2d(net,448,[1,1],scope='Conv2d_0a_1x1')
            branch_2=slim.conv2d(net,384,[3,3],scope='Conv2d_0b_3x3')
            branch_2=tf.concat(3,[slim.conv2d(branch_2,384,[1,3],scope='Conv2d_0c_1x3'),
                                  slim.conv2d(branch_1,384,[3,1],scope='Conv2d_0d_3x1')])

        #第四条路径
        with tf.variable_scope("Branch_3"):
            branch_3=slim.avg_pool2d(net,[3,3],scope='AvgPool_0a_3*3')
            branch_3=slim.conv2d(branch_3,192,[1,1],scope='Conv2d_0b_1x1')

        #输出
        net=tf.concat(3,[branch_0,branch_1,branch_2,branch_3])



